In [ ]:
import earthaccess
import pandas as pd
import geopandas as gpd
import tqdm

In [ ]:
auth = earthaccess.login(strategy="netrc")

In [ ]:
RegionVals = pd.read_pickle("/Users/jmac/Documents/Programming/REU/National-Snow-Model/Data/Processed/RegionVal.pkl")

# S_Sierras = df["S_Sierras"]
# S_Sierras["Region"] = "S_Sierras"
#
# N_Sierras = df["N_Sierras"]
# N_Sierras["Region"] = "N_Sierras"
#
# sierras = pd.concat([S_Sierras, N_Sierras])
# gdf = gpd.GeoDataFrame(sierras, geometry=gpd.points_from_xy(sierras.Long, sierras.Lat))
#
# gdf.to_file("/Users/jmac/Documents/Programming/REU/National-Snow-Model/Data_Processing_Assimilation/sierras.geojson",
#             driver="GeoJSON")
# sierras_bounds = gdf.total_bounds
# sierras_bounds

In [ ]:
# results = query.get()

In [ ]:
# %%time
# files = earthaccess.download(results, local_path="/Users/jmac/Library/CloudStorage/Box-Box/Snow_Modeling_Group/NSM_App_Deployment/National-Snow-Model/Data/VIIRS_SCA")

In [ ]:

def get_links_region(region_gdf, start=2013, stop=2020):
    years = []
    for year in range(2013,2020) :
        bounds = region_gdf.total_bounds
        query = (earthaccess.granule_query()
          .short_name('VNP10A1F')
          .version("1")
          .bounding_box(*bounds)
          .temporal(f"{year}-10-1",f"{year+1}-07-31")
         )

        print(f"{year} hits: {query.hits()}")

        results = query.get(3000)

        urls = [g.data_links(access="on_prem") for g in results]
        years.append({"year":year, "urls": urls})
    return years

    # files = earthaccess.download(results, local_path=f"/Users/jmac/Library/CloudStorage/Box-Box/Snow_Modeling_Group/NSM_App_Deployment/National-Snow-Model/Data/VIIRS_SCA/{year}-{year+1}")

In [ ]:
gdfs = []
for reg in RegionVals:
    df = RegionVals[reg]
    df['region'] = reg
    gdfs.append(df)

In [ ]:
superset = pd.concat(gdfs)
superset = gpd.GeoDataFrame(superset, geometry=gpd.points_from_xy(superset.Long, superset.Lat))

In [ ]:
superset.plot()

In [ ]:
superset.total_bounds

In [ ]:
links = get_links_region(superset)

In [ ]:
links[0]

In [ ]:
for y in links:
    year = links[y]["year"]


In [ ]:
import earthaccess as ea
import shapely
import geopandas as gpd

def calculateGranuleExtent(boundingBox: list[float, float, float, float], day="2019-07-07"):
    """
        Fetches relevant VIIRS granules from NASA's EarthData's CMR API.

        Parameters:
            boundingBox (list[float, float, float, float]): The bounding box of the region of interest.

                lower_left_lon – lower left longitude of the box (west)
                lower_left_lat – lower left latitude of the box (south)
                upper_right_lon – upper right longitude of the box (east)
                upper_right_lat – upper right latitude of the box (north)

        Returns:
            cells (GeoDataFrame): A dataframe containing the horizontal and vertical tile numbers and their boundaries

    """
    auth = ea.login(strategy="netrc")
    if auth is None:
        print("Error logging into Earth Data account.")
        return
    query = (ea.granule_query()
             .short_name('VNP10A1F')  # NPP-SUOMI VIIRS, but JPSS1 VIIRS also exists
             .version("2")  # FIXME v1 supports 2013-onwards, but v2 doesn't work before 2018, update programmatically
             .bounding_box(*boundingBox)
             .temporal(day, day)  # Grab one day's worth of data, we only care about spatial extent
             )

    results = query.get(100)  # The Western CONUS is usually 7, so this is plenty

    cells = []
    for result in results:
        geometry = shapely.geometry.Polygon(
            [(x["Longitude"],x["Latitude"]) for x in
             result["umm"]["SpatialExtent"]["HorizontalSpatialDomain"]["Geometry"]["GPolygons"][0]["Boundary"][
                 "Points"]]
        )
        cell = {
            # "name": 'VNP10A1F',
            # "version": '2',
            "h": result["umm"]["AdditionalAttributes"][1]["Values"],  # HORIZONTAL TILE NUMBER
            "v": result["umm"]["AdditionalAttributes"][2]["Values"],  # VERTICAL TILE NUMBER
            "geometry": geometry
        }
        cells.append(cell)

    geo = gpd.GeoDataFrame(cells, geometry="geometry", crs="EPSG:4326")
    return geo


res = calculateGranuleExtent([-123.34078531, 33.35825379, -105.07803558, 48.97106571])

In [ ]:
res.explore()

In [ ]:
res["geometry"].explore()